# Calcul de la hauteur normale dans un canal

In [1]:
from IPython.display import display, Math, Latex

Par exemple, admettons que l'on doive calculer la hauteur normale dans un canal rectangulaire de largeur $B=2$ m, de pente $i=0{,}1$ %, de rugosité $K=40$ $\mathrm{m}^{1/3}$/s, et pour un débit $Q_0=1$  $\mathrm{m}^3$/s. On doit donc résoudre l'équation $Q(h_n)=Q_0$, soit encore :

\begin{equation}
Q_0=K\sqrt{i}Bh_n\left(\frac{Bh_n}{B+2h_n}\right)^{2/3}.
\end{equation}

Avec la fonction *fsolve*, on va voir ci-dessous que l'on trouve que $h_n=71$ cm ; on note que si l'on avait employé l'approximation pour les canaux larges $h_n=(Q/K/\sqrt{i})^{3/5}$, on trouverait $h_n=57$ cm, ce qui montre qu'il ne faut employer cette approximation que pour des canaux suffisamment larges.

In [4]:
# initialisation
import numpy as np
import matplotlib.pyplot as plt

from scipy.optimize import fsolve

B=2; #largeur
i=1/1000; #slope 
K=40; #coefficient de Strickler
Q0=1; #débit

Q = lambda h: K*(B*h/(B+2*h))**(2/3)*np.sqrt(i)*B*h-Q0

## Résolution avec fsolve

In [5]:
sol=fsolve(Q, [0, 5])
approx=(Q0/B/K/np.sqrt(i))**(3/5)
print(f'hauteur normale {sol[0]:.2} m')
print(f'hauteur normale approchée {approx:.2} m')
 

hauteur normale 0.71 m
hauteur normale approchée 0.57 m


## Développement limité et méthode de la dichotomie

En l'absence de solveur, on peut faire un développement limité. L'équation (1) peut se mettre sous la forme adimensionnelle
\begin{equation}
\left(\frac{Q_0}{K\sqrt{i}B^{8/3}}\right)^{3/2}= \frac{\xi^{5/2}}{1+2\xi},
\end{equation}
avec $\xi=h/B$. Si on néglige les effets de parois comme dans l'approximation de canal large, cela revient à dire que le dénominateur $1+2\xi$ disparaît de l'équation (2), et dans ce cas-là, l'équation(2) devient plus facile à résoudre
\begin{equation}
\left(\frac{Q_0}{K\sqrt{i}B^{8/3}}\right)^{3/2}=  \xi^{5/2} \Rightarrow \xi=\left(\frac{Q_0}{K\sqrt{i}B^{8/3}}\right)^{3/5} = 0,286.
\end{equation}
Cela correspond à la hauteur $h_n=B\xi=57$ cm trouvée précédemment. Cherchons une correction au premier ordre en posant $\xi=\xi_0+\epsilon$ où $\xi_0$ est la valeur que l'on vient de déterminer ($\xi=0{,}286$). Au premier ordre en $\epsilon$, le terme de droite de l'équation \eqref{eq:C5:debit-rectangle2} s'écrit
$$
\frac{\xi^{5/2}}{1+2\xi}=\frac{\xi_0^{5/2}}{2 \xi_0+1}+\frac{\left(6 \xi_0^{5/2}+5 \xi_0^{3/2}\right) }{2 (2 \xi_0+1)^2}\epsilon = 0,0278  + 0,2078  \epsilon.
$${
En substituant le membre de droite par cette approximation linéaire dans l'équation (2), on a une simple égalité à résoudre. On trouve $\epsilon=0{,}077$, et donc $\xi=0{,}286+0{,}077=0{,}363$. La hauteur normale approchée devient alors $h_n=B\xi= 72{,}6$~cm, une valeur proche du résultat (71 cm). On  peut faire une nouvelle itération en considérant que l'on refait un développement limité à l'ordre 1, non plus en $\xi=0{,}286$, mais en $\xi=0,363$. On obtient $\epsilon=-0{,}0078$, soit $h=71$ cm. En deux itérations, on est arrivé au résultat.

L'inconvénient de cette méthode est qu'il y a du calcul algébrique à faire. On peut procéder purement numérique à l'aide de la méthode de la dichotomie. Cette méthode itérative consiste à avoir un encadrement de la solution $f(h)=Q(h)-Q_0=0$ en prenant deux bornes $a_0$ et $b_0$ de l'intervalle telles que $f(a_0)>0$ et $f(b_0)<0$ (ou réciproquement). On prend ensuite le milieu de cet intervalle $c_*=(a_0+b_0)/2$ et on calcule $f(c_*)$. Si $f(c_*)>0$, alors on pose $a_1=c_*$ et $b_1=b_0$ ; réciproquement, i $f(c_*)<0$, alors on pose $b_1=c_*$ et $a_1=a_0$. On itère $N$ fois. Ici on trouve $h_n=71$ cm au bout de $N=8$ itérations. La convergence est assez lente.

In [6]:
def dichotomie(f,a,b,N):
    if f(a)*f(b) >= 0:
        print("Echec : pas de 0 sur l'intervalle")
        return None
    a_n = a
    b_n = b
    for n in range(1,N+1):
        m_n = (a_n + b_n)/2
        f_m_n = f(m_n)
        if f(a_n)*f_m_n < 0:
            a_n = a_n
            b_n = m_n
        elif f(b_n)*f_m_n < 0:
            a_n = m_n
            b_n = b_n
        elif f_m_n == 0:
            print("Solution exacte : ")
            return m_n
        else:
            print("Échec !")
            return None
    return (a_n + b_n)/2

In [7]:
dichotomie(Q,0,5,8)

0.712890625

## Méthode de Newton

Pour résoudre $f(h)=Q(h)-Q_0=0$, nous pouvons aussi employer une méthode qui converge plus rapidement que la dichotomie : c'est la méthode de Newton. L'idée est de partir d'un point $h_0$ et, après avoir calculé la dérivée $f'(h)$, de rechercher l'intersection de la tangente $f(h_0)+(h-h_0)f'(h_0)$ à la courbe $f(h)$ avec l'axe des abscisses. On appelle $h_1$ ce point d'intersection. On itère la méthode :
$$
h_{k+1}=h_k-\frac{f(h_k)}{f'(k)} .
$$
Si tout se passe bien, $h_k$ converge rapidement vers la hauteur normale. Ici, on trouve la bonne solution en trois itérations  quand on prend $h_0=25$ cm.

In [10]:
def newton(f,Df,x0,epsilon,max_iter):

    xn = x0
    for n in range(0,max_iter):
        fxn = f(xn)
        if abs(fxn) < epsilon:
            print('Solution trouvée en',n,'itérations.')
            return xn
        Dfxn = Df(xn)
        if Dfxn == 0:
            print('La dérivée est nulle. Pas de solution.')
            return None
        xn = xn - fxn/Dfxn
    print("Nombre d'itérations dépassé. Pas de solution.")
    return None

In [11]:
dQ = lambda h: K*(B*h/(B+2*h))**(2/3)*np.sqrt(i)*B+2*B*h*np.sqrt(i)*K*(-2*B*h/(B+2*h)**2+B/(B+2*h))/3/(B*h/(B+2*h))**(1/3)
newton(Q,dQ,.25,1e-8,10)

Solution trouvée en 4 itérations.


0.7101688215062145